In [15]:
import pandas as pd
from datetime import datetime
from datetime import timedelta                                   

In [16]:
NGR=pd.read_csv('playerstats.csv')

In [17]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [18]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [19]:
NGR=NGR.loc[(NGR["Advertiser"]!="crmbonus") & (NGR["Advertiser"]!="pine10")]

In [20]:
NGR["Advertiser"].unique()

array(['ally0034', 'ally0004', 'ally0031', 'ally0011', 'ally0032',
       'allyinf0011', 'ally0028', 'rubju300', 'pafeit', 'ally0016',
       'jafe1000', 'ally0002', 'thago', 'jarbasbezerra05', 'ally0021',
       'ally0012', 'jompa', 'rspcom', 'ally0005', 'sixpoint02',
       'ally0036', 'ally0040', 'allyinf0010', 'adriano08', 'ally0003',
       'rspn9', 'ally0023', 'ally0022', 'rspn7', 'rspn10', 'ally0001',
       'ally0037', 'ally00010', 'franber', 'rspn5', 'rspn2', 'ally0015',
       'ally0006', 'Gn8383', 'ally0029', 'ally0007', 'rspn3', 'ally0018',
       'deifes', 'ally0009', 'biar16', 'allyinf009'], dtype=object)

In [21]:
NGR = NGR.loc[(NGR["First deposit"].dt.year == 2024)]

In [22]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [23]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [24]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [25]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [26]:
output

,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,494.37,495.00,11
1,02/2024,-5102.82,3186.00,48
2,03/2024,3989.25,9656.50,94
3,04/2024,8234.82,13340.86,145
4,05/2024,-1129.14,21694.90,298
5,06/2024,-10656.15,29136.23,429
6,07/2024,3391.81,39207.22,509
7,08/2024,10315.11,54518.24,761
8,09/2024,12520.52,52146.35,802
9,10/2024,14550.58,93372.34,1388


In [27]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]
output


,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,494.37,495.00,11
1,02/2024,-5102.82,3186.00,48
2,03/2024,3989.25,9656.50,94
3,04/2024,8234.82,13340.86,145
4,05/2024,-1129.14,21694.90,298
5,06/2024,-10656.15,29136.23,429
6,07/2024,3391.81,39207.22,509
7,08/2024,10315.11,54518.24,761
8,09/2024,12520.52,52146.35,802
9,10/2024,14550.58,93372.34,1388


In [28]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Aplicando a conversão e formatando
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

df['Data'] = "12/12/2024"
df['Ordem'] = range(1, len(df) + 1)


df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD,Data,Ordem
0,01/2024,"R$494,37","R$495,00",11,"$83,05","$83,16",12/12/2024,1
1,02/2024,"R$-5.102,82","R$3.186,00",48,"$-857,27","$535,25",12/12/2024,2
2,03/2024,"R$3.989,25","R$9.656,50",94,"$670,19","$1.622,29",12/12/2024,3
3,04/2024,"R$8.234,82","R$13.340,86",145,"$1.383,45","$2.241,26",12/12/2024,4
4,05/2024,"R$-1.129,14","R$21.694,90",298,"$-189,70","$3.644,74",12/12/2024,5
5,06/2024,"R$-10.656,15","R$29.136,23",429,"$-1.790,23","$4.894,89",12/12/2024,6
6,07/2024,"R$3.391,81","R$39.207,22",509,"$569,82","$6.586,81",12/12/2024,7
7,08/2024,"R$10.315,11","R$54.518,24",761,"$1.732,94","$9.159,06",12/12/2024,8
8,09/2024,"R$12.520,52","R$52.146,35",802,"$2.103,45","$8.760,59",12/12/2024,9
9,10/2024,"R$14.550,58","R$93.372,34",1388,"$2.444,50","$15.686,55",12/12/2024,10


In [29]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats.xlsx",index=False)